---
layout: article
title: Utility of land investing
custom_css: article.css
include_mathjax: true
---

In [ ]:
# import necessary libs (available in jupyter/scipy-notebook docker image)
import os
import pandas as pd 
import numpy as np
import math
import matplotlib.pyplot as plt 
from matplotlib.ticker import FormatStrFormatter

# define watermark 
def add_watermark(ax, x, y):
    ax.text(ax.get_xlim()[0]+ x,
            ax.get_ylim()[0]+ y,
            "ladydragoncapital",
            alpha=0.3, fontsize=16)


# globals
HOME_DIR = '/home/jovyan/_jupyter'
DATA_DIR = os.path.join(HOME_DIR, 'data')

# read in csv data
assets = pd.read_csv(os.path.join(DATA_DIR, 'land_data_05132025.csv'))

In this article, we want to calculate the utility of land investing. This calculation will allow us to not only select from different asset classes, such as land vs. index fund, but also select from multiple options of land. 

Based on our current understanding, the utility of land is a function of monetary return, territory, risk, and liquidity, as below:

$$
U_{land} = \frac{M × T}{R × L}
$$

Where:
+ M: monetary growth
+ T: territory size
+ R: risk, i.e. standard deviation
+ L: liquidity, i.e. time required to liquidate the asset

Monetary growth, $M$, is calculated as the following:

$$
M = w × \frac{\text{Total net gain}}{\text{Total cost}}
$$

$$
= w × \frac{FV + Y - (P + C)}{P+C}
$$

Where: 

+ $w$: weight of monetary growth
+ $FV$: future value of the asset considering the appreciation and carrying cost such as tax and maintenance.
+ $Y$: total revenue from the asset during the ownership, such as rent or profit from agricultural production
+ $P$: initial price (investment) of the asset
+ $C$: fees for owning the asset, such as closing cost for purchasing land, or fees for purchasing stocks.


To calculate the future value of land, we need to consider its annual appreciation, and annual costs such as taxes and maintenance cost.

$$
FV = P×(1+a−t−m)^n 
$$

Where:

+ P: initial investment

+ a: annual appreciation rate

+ t: property tax rate

+ m: maintenance cost rate

+ n: years

The yield of land can come from rent, agricultural production, timber harvesting, etc. It can be calculated as following:

$$
Y = Rv × \frac{(1+g)^n−1}{g}
$$

Where: 

+ Rv: revenue of the first year of owning the land

+ g: annual rental growth rate, if any


So, the monetary utility of land investing is:

$$
M = w × (\frac{P×(1+a−t−m)^n + Rv × \frac{(1+g)^n−1}{g}}{P+C} - 1)
$$

Based on the above equation, in order to increase the monetary utility, we can lower $P$, $C$, $t$, and/or $m$, or increase $a$, $Rv$, or $g$.




Next, we look at the calculation of the territory variable:

$$
T = β × e^{s}
$$

Where:

+ $β$: weight of territory 
+ $s$: area of the land (acres)
  
Calculation of risk:

$$
R = λ × σ^2
$$

Where:
+ $λ$: risk preference or weight of risk
+ $σ$: variance of the market price of the asset

Calculation of liquidity:
$$
L = k × log(D)
$$

Where:

+ $k$: weight of liquidity
+ $D$: days it takes to liquidate the asset

Therefore, the total utility $U$ is:

$$
U_{land} = \frac{M × T}{R × L}
$$

$$
U_{land} = \frac{ w(\frac{P×(1+a−t−m)^n + R × \frac{(1+g)^n−1}{g}}{P+C} - 1) (β × e^{s})}{(λ × σ^2)(k × log(D))}
$$


## Land A vs. Land B ##

Now let's look at an example of using the above utility function to compare two pieces of lands. The table below shows the data of Land A and B. By plugging the data into the above function, we calculated the utilities of the two assets, as shown in Figure 1. 

In [ ]:
# plot the land data in a table
df = assets[:2].copy()
df_transposed = df.set_index("Asset").T.reset_index()  # Moves original headers to a column
df_transposed.columns = ["Asset"] + list(df['Asset'])  # Rename columns

# set the style to a dark theme
plt.style.use("dark_background")

# match website background
plt.rcParams["figure.facecolor"] = "#181818"
plt.rcParams["axes.facecolor"] = "#181818"
plt.rcParams["axes.edgecolor"] = "#181818"

fig, ax = plt.subplots(figsize=(8, 3))  # Adjust size as needed
ax.axis("off")  # Hide axes


# Plot the table with ALL cells (including headers)
table = ax.table(
    cellText=df_transposed.values,  # Use ALL values (including 'Asset' column)
    colLabels=df_transposed.columns,  # Column headers
    loc='center',
    cellLoc='center',
    colColours=['#40466e'] * len(df_transposed.columns)  # Header color
)

# Style headers and cells
for (i, j), cell in table.get_celld().items():
    if i == 0:  # Column headers (top row)
        cell.set_text_props(weight='bold', color='white')
        cell.set_facecolor('#40466e')
    else:
        cell.set_text_props(color='white')
        cell.set_facecolor('black')
    cell.set_edgecolor('gray')



# Style adjustments
table.auto_set_font_size(False)
table.set_fontsize(12)
table.scale(1.2, 1.2)  # Scale cell sizes

plt.title("Table 1. Land A vs. Land B", pad=20, y=1.15)
plt.show()

In [ ]:
# define monetary growth
def monetary_growth(w, P, a, t, m, R, g, n, C):
    ap = P * ((1+a-t-m)**n) 
    re = R * ((1+g)**n - 1)/g
    to = P + C
    return w * (ap + re - to) / to

# define territory
def territory(β, s):
    return β * (np.exp(s))

# define risk
def risk(λ, σ):
    return λ * (σ**2)

# define liquidity
def liquidity(k, D):
    return D**k


# define utility
def utility(w, P, a, t, m, R, g, n, C, β, s, λ, σ, k, D):
    mo = monetary_growth(w, P, a, t, m, R, g, n, C)
    te = territory(β, s)
    ri = risk(λ, σ)
    li = liquidity(k, D)
    return mo * te / ri / li

In [ ]:
# visualize land A vs. land B
fig_count = 1

# data
df['Monetary utility'] = monetary_growth(df['Money weight'], df['Price'], 
                                        df['Appreciation rate'], df['Tax rate'],
                                       df['Maintenance rate'], df['Revenue'], 
                                        df['Revenue growth rate'],
                                       df['Years'], df['Closing cost'])

df['Territory utility'] = territory(df['Territory weight'], df['Acre'])
df['Risk utility'] = risk(df['Risk weight'], df['Stdv'])
df['liquidity utility'] = liquidity(df['Liquidity weight'], df['Liqui days'])

df['Utility'] = utility(df['Money weight'], df['Price'], df['Appreciation rate'], df['Tax rate'],
                        df['Maintenance rate'], df['Revenue'], df['Revenue growth rate'], df['Years'],
                        df['Closing cost'], df['Territory weight'], df['Acre'], df['Risk weight'],
                       df['Stdv'], df['Liquidity weight'], df['Liqui days'])

fig, ax = plt.subplots(figsize=(8, 6))
bars = plt.bar(df['Asset'], df['Utility'], width=0.6)

# Add value labels on top of bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.2f}',
             ha='center', va='bottom', fontsize=12)

# add watermark
add_watermark(ax, 1, 1)

# adding labels and title
plt.xlabel("Asset")
plt.ylabel("Utility")

# set title
plt.suptitle(
    f"Figure {fig_count}. Utility of assets", y=0.0001, fontsize=10
)
fig_count += 1


# displaying the plot
#plt.legend()
plt.show()


Land A's utility is higher than Land B's (481.04 v.s 159.10) due to the bigger land size (1.00 vs. 0.17 acre) and higher monetary growth resulted from higher revenue (300 vs. 100) and revenue growth rate (0.08 vs. 0.05). 

## Utility of index-fund investing ##

Now let's also calculate the utility of index-fund investing. Since there is no territory involved in owning an index fund, the utility of index fund is only a function of three variables: monetary growth, risk, and liquidity, as the following:

$$
U_{index} = \frac{M}{R × L}
$$

$$
U_{index} = \frac{w(\frac{P × (1 + r)^n}{P+C_{index}} - 1)}{(λ × σ^2)(k × log(D))}
$$

Where: 
+ $P$: initial investment
+ $r$: average annual growth rate (including dividends reinvested into the index fund)
+ $n$: years
+ $C_{index}$: fees to purchase and carry index fund
+ $λ$: risk preference
+ $σ$: variance (standard deviation) of the market price of the asset
+ $k$: weight of liquidity
+ $D$: days it takes to liquidate the asset



## Land vs. Index Fund ##

Let's compare the utility of Land A and Index fund C. The data of the two assets are shown in Table 2:

In [ ]:

df2 = assets.iloc[1:3].copy()

# Transpose and reset index
df_transposed2 = df2.set_index('Asset').T.reset_index()
df_transposed2.columns = ['Asset'] + list(df2['Asset'])  # Rename columns

# Create figure and hide axes
fig, ax = plt.subplots(figsize=(8, 3))
ax.axis('off')

# Plot the table with ALL cells (including headers)
table = ax.table(
    cellText=df_transposed2.values,  # Use ALL values (including 'Asset' column)
    colLabels=df_transposed2.columns,  # Column headers
    loc='center',
    cellLoc='center',
    colColours=['#40466e'] * len(df_transposed2.columns)  # Header color
)

# Style headers and cells
for (i, j), cell in table.get_celld().items():
    if i == 0:  # Column headers (top row)
        cell.set_text_props(weight='bold', color='white')
        cell.set_facecolor('#40466e')
    else:
        cell.set_facecolor('black')
    cell.set_edgecolor('gray')


plt.title("Table 2. Land B vs. Index func C", pad=20)
plt.show()

In [ ]:
# define monetary growth
def index_monetary_growth(w, P, r, n, C):
    ap = P * ((1+r)**n) 
    to = P + C
    return w * (ap - to) / to

# define risk
def index_risk(λ, σ):
    return λ * (σ**2)

# define liquidity
def index_liquidity(k, D):
    return D**k


# define utility
def index_utility(w, P, r, n, C, λ, σ, k, D):
    mo = index_monetary_growth(w, P, r, n, C)
    ri = index_risk(λ, σ)
    li = index_liquidity(k, D)
    return mo / ri / li

In [ ]:
# visualize land B vs. Index fund C
df3 = assets[2:3].copy()

# data
df3['Monetary utility'] = index_monetary_growth(df3['Money weight'], df3['Price'], 
                                        df3['Appreciation rate'],
                                       df3['Years'], df3['Closing cost'])

df3['Risk utility'] = index_risk(df3['Risk weight'], df3['Stdv'])
df3['liquidity utility'] = index_liquidity(df3['Liquidity weight'], df3['Liqui days'])

df3['Utility'] = index_utility(df3['Money weight'], df3['Price'], df3['Appreciation rate'], 
                        df3['Years'], df3['Closing cost'], df3['Risk weight'],
                       df3['Stdv'], df3['Liquidity weight'], df3['Liqui days'])


x = ['Land B', 'Index Fund C']
y = [df.loc[1,'Utility'], df3.loc[2,'Utility']]

fig, ax = plt.subplots(figsize=(8, 6))
bars = plt.bar(x, y, width=0.6)

# Add value labels on top of bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.2f}',
             ha='center', va='bottom', fontsize=12)

# add watermark
add_watermark(ax, 1, 40)

# adding labels and title
plt.xlabel("Asset")
plt.ylabel("Utility")

# set title
plt.suptitle(
    f"Figure {fig_count}. Utility of assets", y=0.0001, fontsize=10
)
fig_count += 1


# displaying the plot
#plt.legend()
plt.show()


Figure 2 shows that the utility of Land B is slightly higher than Index Fund C, because despite Index Fund C has a higher monetary growth (1.59 vs. 1.19) and less liquidity days (1 vs. 90), it doesn't have the territory variable and has higher risk (0.15 vs. 0.1 respectively).

## Conclusion ##

The above analysis showed an example on how to use utility to select a more advantageous asset. It can serve a good foundation for further development, such as:

+ adding more variables to the utility function as we understand more about what can be gained from a piece of land

+ adjusting the weights of the variable as our wealth grow and priorities change.